# 概要
TLCから提供されている，LocationIDとZoneをマッピングしているCSVファイルを整形する．

# 入力
[taxi_zone_lookup.csv](https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv)

# 出力
`taxi_zone_lookup_with_coordinates.csv`

In [4]:
from geopy.geocoders import Nominatim
from google.colab import files
import pandas as pd
import time

In [2]:
# CSVファイルをデータフレームとして読み込む
df = pd.read_csv('/content/taxi_zone_lookup.csv')

# データフレームの内容を表示
df

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN


In [3]:
# CSVデータのパス
csv_file_path = '/content/taxi_zone_lookup.csv'

# CSVファイルを読み込む
df_zones = pd.read_csv(csv_file_path)

# Geolocatorの設定
geolocator = Nominatim(user_agent="nyc_taxi_zones")

# 経度と緯度を追加する関数
def get_lat_lon(zone):
    try:
        location = geolocator.geocode(f"{zone}, New York City, NY")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None


# 各ゾーンに対して経度と緯度を取得
df_zones['Latitude'], df_zones['Longitude'] = zip(*df_zones['Zone'].apply(get_lat_lon))
time.sleep(1)  # API呼び出し間の待機時間

In [5]:
# 結果出力
df_zones

,LocationID,Borough,Zone,service_zone,Latitude,Longitude
0,1,EWR,Newark Airport,EWR,NaN,NaN
1,2,Queens,Jamaica Bay,Boro Zone,40.603994,-73.835412
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,NaN,NaN
3,4,Manhattan,Alphabet City,Yellow Zone,40.725102,-73.979583
4,5,Staten Island,Arden Heights,Boro Zone,40.563700,-74.191603
...,...,...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone,40.711900,-74.012527
261,262,Manhattan,Yorkville East,Yellow Zone,40.773514,-73.953631
262,263,Manhattan,Yorkville West,Yellow Zone,NaN,NaN
263,264,Unknown,NaN,NaN,NaN,NaN


In [7]:
# ダウンロード

# データフレームをCSVファイルとして保存
csv_filename = 'taxi_zone_lookup_with_coordinates.csv'
df_zones.to_csv(csv_filename, index=False)

# CSVファイルをダウンロード
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 手修正
ダウンロードされたCSVファイルには経度と緯度がNULL値になっているレコードも含まれることが予想されるため，手動で修正し，NULLを許容しない．

`Zone`カラムがnullもしくは`Outside of NYC`のカラムは削除する．